In [1]:
import syft as sy
sy.requires(">=0.8-beta")

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/teo/PySyft


✅ The installed version of syft==0.8.0b4 matches the requirement >=0.8b0


In [2]:
node = sy.orchestra.launch(name="test-domain-1", processes=1, reset=True)

SQLite Store Path:
!open file:///tmp/7bca415d13ed1ec841f0d0aede098dbb.sqlite

> Starting Worker: test-domain-1 - 7bca415d13ed1ec841f0d0aede098dbb - NodeType.DOMAIN - [<class 'syft.core.node.new.user_service.UserService'>, <class 'syft.core.node.new.metadata_service.MetadataService'>, <class 'syft.core.node.new.action_service.ActionService'>, <class 'syft.core.node.new.test_service.TestService'>, <class 'syft.core.node.new.dataset_service.DatasetService'>, <class 'syft.core.node.new.user_code_service.UserCodeService'>, <class 'syft.core.node.new.request_service.RequestService'>, <class 'syft.core.node.new.data_subject_service.DataSubjectService'>, <class 'syft.core.node.new.network_service.NetworkService'>, <class 'syft.core.node.new.policy_service.PolicyService'>, <class 'syft.core.node.new.message_service.MessageService'>, <class 'syft.core.node.new.project_service.ProjectService'>, <class 'syft.core.node.new.data_subject_member_service.DataSubjectMemberService'>]


In [3]:
node

In [4]:
domain_client = node.login(email="info@openmined.org", password="changethis")

In [5]:
from syft.core.node.new.new_policy import CustomOutputPolicy
@sy.serializable()
class RepeatedCallPolicy(CustomOutputPolicy):
    __canonical_name__ = "RepeatedCallPolicy"
    __version__ = 1
    from typing import List, Dict, Any

    n_calls: int
    downloadable_output_args: List[str]
    state: Dict[Any, Any] = {}
    
    __attr_allowlist__ = [
        "n_calls",
        "downloadable_output_args",
    ]

    def __init__(self, n_calls=1, downloadable_output_args=None):
        super().__init__(n_calls=n_calls, downloadable_output_args=downloadable_output_args)
        self.n_calls = n_calls + 1
        self.downloadable_output_args = downloadable_output_args if not None else []
        self.state = {"counts": 0}

    def public_state(self):
        return self.state["counts"]
        
    def apply_output(self, context, outputs):
        output_dict = {}
        results_dict = outputs.syft_action_data        
        if self.state["counts"] < self.n_calls:
            for output_arg in self.downloadable_output_args:
                output_dict[output_arg] = results_dict[output_arg]

            self.state["counts"] += 1
        else:
            return None

        return output_dict

In [6]:
policy = RepeatedCallPolicy(n_calls=1, downloadable_output_args=['y'])
print(policy.init_args, policy.init_kwargs)
a_obj = sy.ActionObject.from_obj({'y': [1,2,3]})
policy.apply_output(None, a_obj)

{} {'n_calls': 1, 'downloadable_output_args': ['y']}


{'y': [1, 2, 3]}

In [7]:
import numpy as np
x = np.array([1,2,3])
x_pointer = sy.ActionObject.from_obj(x)
domain_client.api.services.action.save(x_pointer)

<class 'syft.core.node.new.response.SyftSuccess'>: <class 'syft.core.node.new.numpy.NumpyArrayObject'> saved

In [8]:
obj = RepeatedCallPolicy(n_calls=1, downloadable_output_args=['y'])
obj

```python
class RepeatedCallPolicy:
  id: str = bf3d4ece21bb4590930f38a87daa9ede
  output_history: str = []
  outputs: str = []
  node_uid: str = None
  init_args: str = {}
  init_kwargs: str = {'n_calls': 1, 'downloadable_output_args': ['y']}
  n_calls: str = 2
  downloadable_output_args: str = ['y']
  state: str = {'counts': 0}
  List: str = typing.List
  Dict: str = typing.Dict
  Any: str = typing.Any

```

In [9]:
obj

```python
class RepeatedCallPolicy:
  id: str = bf3d4ece21bb4590930f38a87daa9ede
  output_history: str = []
  outputs: str = []
  node_uid: str = None
  init_args: str = {}
  init_kwargs: str = {'n_calls': 1, 'downloadable_output_args': ['y']}
  n_calls: str = 2
  downloadable_output_args: str = ['y']
  state: str = {'counts': 0}
  List: str = typing.List
  Dict: str = typing.Dict
  Any: str = typing.Any

```

In [10]:
from syft.core.node.new.new_policy import ExactMatch
@sy.syft_function(
    input_policy=ExactMatch(x=x_pointer),
    output_policy=RepeatedCallPolicy(n_calls=10, downloadable_output_args=['y']),
)
def func(x):
    return {"y": x+1}

{'n_calls': 10, 'downloadable_output_args': ['y']}


In [11]:
domain_client.api.services.code.request_code_execution(func)

what kind of output_policy syft.core.node.new.new_policy.UserPolicy <class 'syft.core.node.new.new_policy.UserPolicy'>


```python
class Request:
  id: str = 281b5caf338a4437953ec61bef2d4f42
  requesting_user_verify_key: str = aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f
  approving_user_verify_key: str = None
  request_time: str = 2023-03-28 17:31:25
  approval_time: str = None
  status: str = RequestStatus.PENDING
  node_uid: str = 7bca415d13ed1ec841f0d0aede098dbb
  request_hash: str = "c9d300f849565b4a0e3e5852b93fdf1793f69539823557d102021d9741362da5"
  changes: str = [syft.core.node.new.request.UserCodeStatusChange, syft.core.node.new.request.EnumMutation]

```

In [12]:
request = domain_client.notifications[-1].link
change = func = request.changes[-1]
change

```python
class EnumMutation:
  id: str = f6f0c800ceff4cb6a33fdf492b3b4404
  linked_obj: str = <<class 'syft.core.node.new.new_policy.UserPolicy'>: 462fc858b3f54f52a30e565a27596f17@<Node: 7bca415d13ed1ec841f0d0aede098dbb>
  attr_name: str = "status"
  value: str = UserPolicyStatus.APPROVED
  match_type: str = True
  enum_type: str = <enum 'UserPolicyStatus'>

```

In [13]:
request

```python
class Request:
  id: str = 281b5caf338a4437953ec61bef2d4f42
  requesting_user_verify_key: str = aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f
  approving_user_verify_key: str = None
  request_time: str = 2023-03-28 17:31:25
  approval_time: str = None
  status: str = RequestStatus.PENDING
  node_uid: str = 7bca415d13ed1ec841f0d0aede098dbb
  request_hash: str = "c9d300f849565b4a0e3e5852b93fdf1793f69539823557d102021d9741362da5"
  changes: str = [syft.core.node.new.request.UserCodeStatusChange, syft.core.node.new.request.EnumMutation]

```

In [14]:
request.approve()

<class 'syft.core.node.new.response.SyftSuccess'>: Request 281b5caf338a4437953ec61bef2d4f42 changes applied

In [15]:
func = request.changes[-2].link
func

```python
class UserCode:
  id: str = 0ed77cc859ba4ac28f1d479b3512ddb4
  user_verify_key: str = aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f
  raw_code: str = "@sy.syft_function(
    input_policy=ExactMatch(x=x_pointer),
    output_policy=RepeatedCallPolicy(n_calls=10, downloadable_output_args=['y']),
)
def func(x):
    return {"y": x+1}
"
  input_policy: str = syft.core.node.new.new_policy.ExactMatch
  input_policy_state: str = b''
  output_policy: str = syft.core.node.new.new_policy.UserPolicy
  output_policy_state: str = b''
  parsed_code: str = "

def user_func_func_aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f_e14bece1fc07fb142d5dd8365a4fdd3b6eeaf6f9826e22582e41e972340a190e(x):

    def func(x):
        return {'y': (x + 1)}
    result = func(x=x)
    return result
"
  service_func_name: str = "func"
  unique_func_name: str = "user_func_func_aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f_e14bece1fc07fb142d5dd8365a4fdd3b6eeaf6f9826e22582e41e972340a190e"
  user_unique_func_name: str = "user_func_func_aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f"
  code_hash: str = "e14bece1fc07fb142d5dd8365a4fdd3b6eeaf6f9826e22582e41e972340a190e"
  signature: str = (x) -> Dict[str, Any]
  status: str = {NodeView(node_name='test-domain-1', verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f): <UserCodeStatus.EXECUTE: 'execute'>}
  input_kwargs: str = ['x']
  outputs: str = []
  input_policy_init_args: str = None
  output_policy_init_args: str = {'n_calls': 10, 'downloadable_output_args': ['y']}
  enclave_metadata: str = None

```

In [16]:
result = func.unsafe_function(x=x_pointer)
result

Exception detected in make_action_side_effect 'int' object has no attribute 'syft_lineage_id'


,key,type,id
0,y,"array([2, 3, 4])",556eb84cfe6540719a11699a024ac820


In [17]:
result

,key,type,id
0,y,"array([2, 3, 4])",556eb84cfe6540719a11699a024ac820


In [18]:
final_result = request.accept_by_depositing_result(result) 
final_result

<class 'syft.core.node.new.response.SyftSuccess'>: Request 281b5caf338a4437953ec61bef2d4f42 changes applied

In [19]:
res = domain_client.api.services.code.func(x=x_pointer)
res

Ok({'y': array([2, 3, 4])})
Exec context
Nope it didnt work


{'y': array([2, 3, 4])}

In [20]:
assert (res["y"] == np.array([2, 3, 4])).all()

Exception detected in make_action_side_effect 'str' object has no attribute 'syft_lineage_id'
Exception detected in make_action_side_effect 'numpy.ndarray' object has no attribute 'syft_lineage_id'


In [21]:
assert set(res.syft_action_data.keys()) == set(list("y"))

In [22]:
domain_client.api.services.code.get_all()[0].output_policy

```python
class UserPolicy:
  id: str = 462fc858b3f54f52a30e565a27596f17
  node_uid: str = 7bca415d13ed1ec841f0d0aede098dbb
  user_verify_key: str = aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f
  raw_code: str = "class RepeatedCallPolicy(CustomOutputPolicy):
    __canonical_name__ = "RepeatedCallPolicy"
    __version__ = 1
    from typing import List, Dict, Any

    n_calls: int
    downloadable_output_args: List[str]
    state: Dict[Any, Any] = {}
    
    __attr_allowlist__ = [
        "n_calls",
        "downloadable_output_args",
    ]

    def __init__(self, n_calls=1, downloadable_output_args=None):
        super().__init__(n_calls=n_calls, downloadable_output_args=downloadable_output_args)
        self.n_calls = n_calls + 1
        self.downloadable_output_args = downloadable_output_args if not None else []
        self.state = {"counts": 0}

    def public_state(self):
        return self.state["counts"]
        
    def apply_output(self, context, outputs):
        output_dict = {}
        results_dict = outputs.syft_action_data        
        if self.state["counts"] < self.n_calls:
            for output_arg in self.downloadable_output_args:
                output_dict[output_arg] = results_dict[output_arg]

            self.state["counts"] += 1
        else:
            return None

        return output_dict
"
  parsed_code: str = "

@serializable(recursive_serde=True)
class RepeatedCallPolicy_aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f_6de8630e91e629badda5fda25287ba6e9ceeeeafe38ea4f398bb58f460bdcf34(CustomOutputPolicy):
    __canonical_name__ = 'RepeatedCallPolicy'
    __version__ = 1
    from typing import List, Dict, Any
    n_calls: int
    downloadable_output_args: List[str]
    state: Dict[(Any, Any)] = {}
    __attr_allowlist__ = ['n_calls', 'downloadable_output_args']

    def __init__(self, n_calls=1, downloadable_output_args=None):
        super().__init__(n_calls=n_calls, downloadable_output_args=downloadable_output_args)
        self.n_calls = (n_calls + 1)
        self.downloadable_output_args = (downloadable_output_args if (not None) else [])
        self.state = {'counts': 0}

    def public_state(self):
        return self.state['counts']

    def apply_output(self, context, outputs):
        output_dict = {}
        results_dict = outputs.syft_action_data
        if (self.state['counts'] < self.n_calls):
            for output_arg in self.downloadable_output_args:
                output_dict[output_arg] = results_dict[output_arg]
            self.state['counts'] += 1
        else:
            return None
        return output_dict
"
  signature: str = (n_calls, downloadable_output_args)
  class_name: str = "RepeatedCallPolicy"
  unique_name: str = "RepeatedCallPolicy_aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f_6de8630e91e629badda5fda25287ba6e9ceeeeafe38ea4f398bb58f460bdcf34"
  code_hash: str = "6de8630e91e629badda5fda25287ba6e9ceeeeafe38ea4f398bb58f460bdcf34"
  status: str = UserPolicyStatus.SUBMITTED
  state_type: str = None
  policy_version: str = 1

```

In [23]:
domain_client.api.services.policy.get_all()

,type,id
0,syft.core.node.new.new_policy.UserPolicy,462fc858b3f54f52a30e565a27596f17


In [24]:
output_policy = domain_client.api.services.policy.get_all()[0]

In [25]:
sy.orchestra.land("test-domain-1")

SQLite Store Path:
!open file:///tmp/7bca415d13ed1ec841f0d0aede098dbb.sqlite

> Starting Worker: test-domain-1 - 7bca415d13ed1ec841f0d0aede098dbb - NodeType.DOMAIN - [<class 'syft.core.node.new.user_service.UserService'>, <class 'syft.core.node.new.metadata_service.MetadataService'>, <class 'syft.core.node.new.action_service.ActionService'>, <class 'syft.core.node.new.test_service.TestService'>, <class 'syft.core.node.new.dataset_service.DatasetService'>, <class 'syft.core.node.new.user_code_service.UserCodeService'>, <class 'syft.core.node.new.request_service.RequestService'>, <class 'syft.core.node.new.data_subject_service.DataSubjectService'>, <class 'syft.core.node.new.network_service.NetworkService'>, <class 'syft.core.node.new.policy_service.PolicyService'>, <class 'syft.core.node.new.message_service.MessageService'>, <class 'syft.core.node.new.project_service.ProjectService'>, <class 'syft.core.node.new.data_subject_member_service.DataSubjectMemberService'>]
